# Sales Visualizations and Next-Year Forecast (Linear Regression)

This notebook:
- builds required sales visualizations,
- compares **LinearRegression-only** variants,
- selects the best out-of-sample model by validation R2,
- predicts next-year sales using `REAL_DATA.csv` shifted by +1 year.


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

np.random.seed(42)
sns.set_theme(style='whitegrid', context='talk')
plt.rcParams['figure.figsize'] = (12, 6)


## 1) Load and Clean Data


In [ ]:
sales_path = 'data/Copy_of_sales-sales.csv'
real_path = 'data/REAL_DATA.csv'

sales_df = pd.read_csv(sales_path)
real_df = pd.read_csv(real_path)

for df in (sales_df, real_df):
    for col in ['Unnamed: 0', 'index']:
        if col in df.columns:
            df.drop(columns=col, inplace=True)

sales_df['date'] = pd.to_datetime(sales_df['date'], errors='coerce')
real_df['date'] = pd.to_datetime(real_df['date'], dayfirst=True, errors='coerce')

required_features = [
    'store_ID', 'day_of_week', 'date', 'nb_customers_on_day',
    'open', 'promotion', 'state_holiday', 'school_holiday'
]

missing_sales_features = [c for c in required_features if c not in sales_df.columns]
missing_real_features = [c for c in required_features if c not in real_df.columns]
assert not missing_sales_features, f'Missing in sales_df: {missing_sales_features}'
assert not missing_real_features, f'Missing in real_df: {missing_real_features}'
assert 'sales' in sales_df.columns, "sales target not found in Copy_of_sales-sales.csv"
assert sales_df['sales'].notna().all(), "sales contains null values"
assert sales_df['date'].notna().all(), "sales_df date parse produced nulls"
assert real_df['date'].notna().all(), "real_df date parse produced nulls"

print('sales_df shape:', sales_df.shape)
print('real_df shape:', real_df.shape)
print('sales date range:', sales_df['date'].min(), 'to', sales_df['date'].max())
print('real date range :', real_df['date'].min(), 'to', real_df['date'].max())


## 2) Shared Feature Engineering


In [ ]:
def add_time_features(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out['year'] = out['date'].dt.year
    out['month'] = out['date'].dt.month
    out['day'] = out['date'].dt.day
    out['weekofyear'] = out['date'].dt.isocalendar().week.astype(int)
    out['quarter'] = out['date'].dt.quarter
    out['is_weekend'] = out['day_of_week'].isin([6, 7]).astype(int)
    out['state_holiday_flag'] = (out['state_holiday'].astype(str) != '0').astype(int)
    return out


def add_interactions(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out['customers_x_promo'] = out['nb_customers_on_day'] * out['promotion']
    out['open_x_promo'] = out['open'] * out['promotion']
    out['customers_x_open'] = out['nb_customers_on_day'] * out['open']
    return out

sales_feat = add_time_features(sales_df)


## 3) Required Visual: Histogram of Sales by Year


In [ ]:
yearly_sales = sales_feat.groupby('year', as_index=False)['sales'].sum()
print(yearly_sales)

plt.figure(figsize=(10, 6))
sns.histplot(yearly_sales['sales'], bins=8, kde=True, color='steelblue')
plt.title('Histogram of Annual Sales Totals')
plt.xlabel('Annual Sales')
plt.ylabel('Count')
plt.tight_layout()
plt.show()


## 4) Required Visual: Best Performing Stores Over the Years


In [ ]:
store_year = sales_feat.groupby(['store_ID', 'year'], as_index=False)['sales'].sum()
store_total = store_year.groupby('store_ID', as_index=False)['sales'].sum().sort_values('sales', ascending=False)
top10_stores = store_total.head(10)['store_ID']
store_year_top10 = store_year[store_year['store_ID'].isin(top10_stores)]

plt.figure(figsize=(14, 7))
sns.lineplot(data=store_year_top10, x='year', y='sales', hue='store_ID', marker='o')
plt.title('Top 10 Stores: Sales Trend Over Years')
plt.xlabel('Year')
plt.ylabel('Sales')
plt.legend(title='Store ID', bbox_to_anchor=(1.02, 1), loc='upper left')
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 6))
sns.barplot(data=store_total.head(10), x='store_ID', y='sales', palette='viridis')
plt.title('Top 10 Stores by Total Sales')
plt.xlabel('Store ID')
plt.ylabel('Total Sales')
plt.tight_layout()
plt.show()


## 5) Modeling: LinearRegression Variants (No Ridge)


In [ ]:
sales_model_df = add_time_features(sales_df)

train_df = sales_model_df[sales_model_df['date'] <= pd.Timestamp('2014-12-31')].copy()
val_df = sales_model_df[sales_model_df['date'] >= pd.Timestamp('2015-01-01')].copy()
assert len(train_df) > 0 and len(val_df) > 0, 'Train/validation split failed.'

numeric_base = [
    'nb_customers_on_day', 'year', 'month', 'day', 'weekofyear',
    'quarter', 'is_weekend'
]

categorical_cols = [
    'store_ID', 'day_of_week', 'open', 'promotion',
    'state_holiday', 'school_holiday'
]

variant_specs = {
    'A_numeric_only': {
        'numeric': numeric_base + ['store_ID', 'day_of_week', 'open', 'promotion', 'school_holiday', 'state_holiday_flag'],
        'categorical': [],
        'use_interactions': False,
    },
    'B_numeric_plus_ohe': {
        'numeric': numeric_base,
        'categorical': categorical_cols,
        'use_interactions': False,
    },
    'C_plus_interactions': {
        'numeric': numeric_base + ['customers_x_promo', 'open_x_promo', 'customers_x_open'],
        'categorical': categorical_cols,
        'use_interactions': True,
    },
}

results = []
trained_models = {}
for name, spec in variant_specs.items():
    tr = train_df.copy()
    va = val_df.copy()
    if spec['use_interactions']:
        tr = add_interactions(tr)
        va = add_interactions(va)

    X_train = tr[spec['numeric'] + spec['categorical']]
    y_train = tr['sales']
    X_val = va[spec['numeric'] + spec['categorical']]
    y_val = va['sales']

    numeric_transformer = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()),
    ])

    if spec['categorical']:
        categorical_transformer = Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore')),
        ])
        preprocessor = ColumnTransformer([
            ('num', numeric_transformer, spec['numeric']),
            ('cat', categorical_transformer, spec['categorical']),
        ])
    else:
        preprocessor = ColumnTransformer([
            ('num', numeric_transformer, spec['numeric']),
        ])

    model = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', LinearRegression()),
    ])

    model.fit(X_train, y_train)
    val_pred = np.clip(model.predict(X_val), 0, None)
    r2 = r2_score(y_val, val_pred)
    mae = mean_absolute_error(y_val, val_pred)

    results.append({'variant': name, 'r2': r2, 'mae': mae})
    trained_models[name] = (model, spec)

results_df = pd.DataFrame(results).sort_values('r2', ascending=False).reset_index(drop=True)
display(results_df)

best_variant = results_df.loc[0, 'variant']
best_model, best_spec = trained_models[best_variant]
print('Best variant:', best_variant)


## 6) Validation Diagnostic Plot


In [ ]:
val_diag = val_df.copy()
if best_spec['use_interactions']:
    val_diag = add_interactions(val_diag)

X_val_best = val_diag[best_spec['numeric'] + best_spec['categorical']]
val_diag['pred_sales'] = np.clip(best_model.predict(X_val_best), 0, None)

actual_monthly = val_diag.groupby(val_diag['date'].dt.to_period('M'))['sales'].sum().reset_index()
pred_monthly = val_diag.groupby(val_diag['date'].dt.to_period('M'))['pred_sales'].sum().reset_index()
actual_monthly['date'] = actual_monthly['date'].dt.to_timestamp()
pred_monthly['date'] = pred_monthly['date'].dt.to_timestamp()

plt.figure(figsize=(13, 6))
sns.lineplot(data=actual_monthly, x='date', y='sales', label='Actual', marker='o')
sns.lineplot(data=pred_monthly, x='date', y='pred_sales', label='Predicted', marker='o')
plt.title('Validation Window: Actual vs Predicted Monthly Sales')
plt.xlabel('Month')
plt.ylabel('Sales')
plt.tight_layout()
plt.show()


## 7) Forecast Next Year from REAL_DATA (+1 Year Shift)


In [ ]:
future_df = real_df.copy()
future_df['date'] = future_df['date'] + pd.DateOffset(years=1)
future_df = add_time_features(future_df)
if best_spec['use_interactions']:
    future_df = add_interactions(future_df)

X_future = future_df[best_spec['numeric'] + best_spec['categorical']]
future_df['predicted_sales'] = np.clip(best_model.predict(X_future), 0, None)
future_df['year'] = future_df['date'].dt.year

forecast_daily = future_df[['store_ID', 'date', 'year', 'predicted_sales']].copy()
forecast_yearly = (
    forecast_daily
    .groupby(['store_ID', 'year'], as_index=False)['predicted_sales']
    .sum()
    .sort_values('predicted_sales', ascending=False)
)

forecast_yearly.to_csv('forecast_next_year_linear.csv', index=False)
print('Saved forecast_next_year_linear.csv')
display(forecast_yearly.head(10))


## 8) Forecast Visualizations


In [ ]:
monthly_forecast = (
    forecast_daily
    .groupby(forecast_daily['date'].dt.to_period('M'))['predicted_sales']
    .sum()
    .reset_index()
)
monthly_forecast['date'] = monthly_forecast['date'].dt.to_timestamp()

plt.figure(figsize=(13, 6))
sns.lineplot(data=monthly_forecast, x='date', y='predicted_sales', marker='o', color='teal')
plt.title('Predicted Monthly Sales for Next Year')
plt.xlabel('Month')
plt.ylabel('Predicted Sales')
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 6))
top15 = forecast_yearly.head(15)
sns.barplot(data=top15, x='store_ID', y='predicted_sales', palette='magma')
plt.title('Top 15 Stores by Predicted Next-Year Sales')
plt.xlabel('Store ID')
plt.ylabel('Predicted Sales')
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 6))
sns.histplot(forecast_daily['predicted_sales'], bins=50, kde=True, color='darkorange')
plt.title('Distribution of Daily Predicted Sales (Next Year)')
plt.xlabel('Predicted Daily Sales')
plt.ylabel('Count')
plt.tight_layout()
plt.show()


## 9) Extra Useful Visuals


In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(data=sales_feat, x='promotion', y='sales')
plt.title('Sales Distribution by Promotion Flag')
plt.xlabel('Promotion')
plt.ylabel('Sales')
plt.tight_layout()
plt.show()

sample_n = min(50000, len(sales_feat))
sample_df = sales_feat.sample(sample_n, random_state=42)
plt.figure(figsize=(11, 6))
sns.regplot(
    data=sample_df,
    x='nb_customers_on_day',
    y='sales',
    scatter_kws={'alpha': 0.2, 's': 12},
    line_kws={'color': 'red'}
)
plt.title('Customers vs Sales (Sample)')
plt.xlabel('Number of Customers')
plt.ylabel('Sales')
plt.tight_layout()
plt.show()

seasonality = (
    sales_feat
    .groupby(['month', 'day_of_week'], as_index=False)['sales']
    .mean()
    .pivot(index='month', columns='day_of_week', values='sales')
)

plt.figure(figsize=(11, 7))
sns.heatmap(seasonality, cmap='YlGnBu', annot=False)
plt.title('Average Sales Heatmap: Month vs Day of Week')
plt.xlabel('Day of Week')
plt.ylabel('Month')
plt.tight_layout()
plt.show()


## 10) Validation and Sanity Checks


In [ ]:
orig_dates = real_df['date'].sort_values().reset_index(drop=True)
shifted_dates = future_df['date'].sort_values().reset_index(drop=True)
delta_days = (shifted_dates - orig_dates).dt.days
assert delta_days.isin([365, 366]).all(), 'Shift must be exactly +1 year (365/366 days).'
assert {'store_ID', 'year', 'predicted_sales'}.issubset(set(forecast_yearly.columns))
assert len(forecast_yearly) > 0
assert (forecast_daily['predicted_sales'] >= 0).all()
print('All checks passed.')
